In [1]:
!pip install -q sklearn

In [2]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [4]:
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials
# # Authenticate and create the PyDrive client.
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [143]:
# downloaded = drive.CreateFile({'id':'1ibpycTdIFp0RH8UNdc0bX7v6v2PO2u4Ip8rByTWwZlg'}) 
# downloaded.GetContentFile('ytrain.csv')  
dftrain = pd.read_csv('/content/gdrive/MyDrive/dftrain.csv')
dfeval = pd.read_csv('/content/gdrive/MyDrive/dfeval.csv')
# dftrain = pd.read_csv('/content/gdrive/MyDrive/FFL/dftrain2.csv')
# dfeval = pd.read_csv('/content/gdrive/MyDrive/FFL/dfeval2.csv')
dftrain = dftrain.drop(['Unnamed: 0'], axis=1)
dfeval = dfeval.drop(['Unnamed: 0'], axis=1)
# dftrain = dftrain.drop(['long'], axis=1)
# dfeval = dfeval.drop(['long'], axis=1)
# dftrain = dftrain.drop(['Year'], axis=1)
# dfeval = dfeval.drop(['Year'], axis=1)
# dftrain = dftrain.drop(['lat'], axis=1)
# dfeval = dfeval.drop(['lat'], axis=1)
# dftrain = dftrain.drop(['elevation'], axis=1)
# dfeval = dfeval.drop(['elevation'], axis=1)
# dftrain = dftrain.head(1000)
# dfeval = dfeval.head(1000)
y_train = dftrain.pop('win')
y_eval = dfeval.pop('win')

In [144]:
dftrain

,Date,Tm,p_Cmp,p_Att,p_Yds,p_TD,Int,Sk,Yds_lost,p_Lng,p_Rate,ru_Att,ru_Yds,ru_TD,ru_Lng,re_targ,Rec,re_Yds,re_TD,re_Lng,fmb,fmb_loss,temp,wind,avg_p_Cmp,avg_p_Att,avg_p_Yds,avg_p_TD,avg_Int,avg_Sk,avg_Yds_lost,avg_p_Lng,avg_p_Rate,avg_ru_Att,avg_ru_Yds,avg_ru_TD,avg_ru_Lng,avg_re_targ,avg_Rec,avg_re_Yds,avg_re_TD,avg_re_Lng,avg_fmb,avg_fmb_loss,last_p_Cmp,last_p_Att,last_p_Yds,last_p_TD,last_Int,last_Sk,last_Yds_lost,last_p_Lng,last_p_Rate,last_ru_Att,last_ru_Yds,last_ru_TD,last_ru_Lng,last_re_targ,last_Rec,last_re_Yds,last_re_TD,last_re_Lng,last_fmb,last_fmb_loss,lat,long,elevation
0,10/8/2020,Tampa Bay Buccaneers,25,41,253,1,0,3,20,35,86.7,20,106,0,38,41,25,253,1,125,2,1,0,0,24.333333,38.833333,264.000000,2.000000,0.833333,1.333333,7.833333,41.000000,89.600000,18.800000,74.800000,0.366667,22.533333,21.266667,13.800000,126.566667,2.033333,60.766667,1.166667,0.666667,30,46,369,5,1,0,0,48,117.0,26,112,0,25,24,17,193,3,100,0,0,358.85943,-677.04102,27.0
1,10/1/1979,Green Bay Packers,17,29,206,1,2,1,7,29,147.0,35,118,3,28,0,17,206,1,107,0,0,468,63,14.000000,23.200000,156.200000,0.600000,0.600000,0.000000,0.000000,32.800000,78.820000,26.033333,90.166667,1.900000,25.766667,0.000000,13.616667,154.983333,0.733333,85.316667,0.000000,0.000000,10,15,108,1,0,0,0,20,109.9,36,135,3,33,0,12,134,1,62,0,0,357.86970,-943.82460,14500.8
2,9/30/1990,Detroit Lions,17,34,200,1,0,1,6,40,78.1,26,144,2,30,0,17,200,1,100,0,0,504,0,14.250000,26.750000,169.750000,1.250000,0.750000,2.750000,15.750000,28.500000,76.225000,21.750000,113.000000,1.250000,34.000000,0.000000,16.666667,201.083333,1.500000,91.833333,0.000000,0.000000,17,26,169,1,0,3,17,19,96.5,23,149,1,47,0,19,213,1,103,0,0,385.66670,-903.66670,1890.0
3,12/6/1998,Denver Broncos,22,32,400,2,1,2,17,58,119.3,29,93,3,29,32,22,400,2,218,0,0,225,72,15.066667,24.900000,210.683333,1.666667,0.666667,1.250000,9.500000,37.100000,96.550000,27.333333,134.716667,1.616667,44.266667,28.951299,17.707576,242.304762,2.182684,99.300000,0.000000,0.000000,19,34,239,4,3,1,13,37,80.4,29,92,0,38,37,21,247,5,114,0,0,455.46270,-900.37200,2618.0
4,11/12/1978,Green Bay Packers,7,26,88,1,2,3,12,16,140.1,17,66,1,50,0,7,88,1,63,0,0,310,140,10.636364,20.636364,128.363636,0.727273,1.000000,0.000000,0.000000,29.909091,66.000000,33.662626,139.741414,1.181818,51.577778,0.000000,12.582395,154.029437,0.819625,89.589033,0.000000,0.000000,18,33,220,0,3,1,9,50,37.4,33,169,0,59,0,18,220,0,131,0,0,337.39000,-843.89000,10500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20454,12/16/2001,Cleveland Browns,21,30,184,0,1,3,11,44,72.1,15,47,0,17,30,21,184,0,115,0,0,380,90,16.692308,28.307692,180.846154,1.000000,1.153846,3.076923,20.000000,34.230769,75.653846,17.576923,53.423077,0.403846,24.903846,22.771368,13.822650,154.692308,0.894231,75.985043,0.000000,0.000000,20,39,244,0,3,3,17,36,38.8,11,53,0,33,23,15,189,0,90,0,0,329.46551,-990.89419,33.0
20455,11/24/1985,Indianapolis Colts,10,24,116,0,1,3,30,41,94.2,23,125,1,40,0,10,116,0,83,0,0,154,98,17.666667,35.750000,220.333333,1.000000,1.750000,3.750000,27.000000,60.583333,131.633333,29.003030,136.511364,1.246970,46.720455,0.000000,8.047727,90.913636,0.363636,58.213636,0.000000,0.000000,24,50,356,1,3,7,42,109,109.7,19,143,0,62,0,6,66,0,57,0,0,312.80000,-676.13336,1551.2
20456,9/18/2016,Arizona Cardinals,18,34,315,3,0,1,4,69,164.5,30,105,1,25,34,18,315,3,175,0,0,864,0,24.700000,45.800000,305.100000,2.000000,2.400000,3.000000,14.400000,58.400000,114.380000,24.500000,81.600000,0.900000,27.100000,39.816667,22.750000,289.466667,2.100000,142.750000,0.800000,0.400000,32,55,355,2,2,5,27,60,123.7,19,92,1,48,40,26,304,3,168,1,0,327.57200,-970.73600,1634.0
20457,10/19/1997,Washington Redskins,16,31,228,2,3,1,8,37,57.7,18,92,0,49,29,16,228,2,101,0,0,495,36,15.428571,30.428571,217.857143,1.

In [145]:
dftrain.columns

Index(['Date', 'Tm', 'p_Cmp', 'p_Att', 'p_Yds', 'p_TD', 'Int', 'Sk',
       'Yds_lost', 'p_Lng', 'p_Rate', 'ru_Att', 'ru_Yds', 'ru_TD', 'ru_Lng',
       're_targ', 'Rec', 're_Yds', 're_TD', 're_Lng', 'fmb', 'fmb_loss',
       'temp', 'wind', 'avg_p_Cmp', 'avg_p_Att', 'avg_p_Yds', 'avg_p_TD',
       'avg_Int', 'avg_Sk', 'avg_Yds_lost', 'avg_p_Lng', 'avg_p_Rate',
       'avg_ru_Att', 'avg_ru_Yds', 'avg_ru_TD', 'avg_ru_Lng', 'avg_re_targ',
       'avg_Rec', 'avg_re_Yds', 'avg_re_TD', 'avg_re_Lng', 'avg_fmb',
       'avg_fmb_loss', 'last_p_Cmp', 'last_p_Att', 'last_p_Yds', 'last_p_TD',
       'last_Int', 'last_Sk', 'last_Yds_lost', 'last_p_Lng', 'last_p_Rate',
       'last_ru_Att', 'last_ru_Yds', 'last_ru_TD', 'last_ru_Lng',
       'last_re_targ', 'last_Rec', 'last_re_Yds', 'last_re_TD', 'last_re_Lng',
       'last_fmb', 'last_fmb_loss', 'lat', 'long', 'elevation'],
      dtype='object')

In [146]:
y_train

0        0
1        1
2        0
3        1
4        0
        ..
20454    0
20455    0
20456    1
20457    0
20458    0
Name: win, Length: 20459, dtype: int64

In [147]:
CATEGORICAL_COLUMNS = ['Date', 'Tm']
NUMERIC_COLUMNS = ['p_Cmp', 'p_Att', 'p_Yds', 'p_TD', 'Int', 'Sk',
       'Yds_lost', 'p_Lng', 'p_Rate', 'ru_Att', 'ru_Yds', 'ru_TD', 'ru_Lng',
       're_targ', 'Rec', 're_Yds', 're_TD', 're_Lng', 'fmb', 'fmb_loss',
       'temp', 'wind', 'avg_p_Cmp', 'avg_p_Att', 'avg_p_Yds', 'avg_p_TD',
       'avg_Int', 'avg_Sk', 'avg_Yds_lost', 'avg_p_Lng', 'avg_p_Rate',
       'avg_ru_Att', 'avg_ru_Yds', 'avg_ru_TD', 'avg_ru_Lng', 'avg_re_targ',
       'avg_Rec', 'avg_re_Yds', 'avg_re_TD', 'avg_re_Lng', 'avg_fmb',
       'avg_fmb_loss', 'last_p_Cmp', 'last_p_Att', 'last_p_Yds', 'last_p_TD',
       'last_Int', 'last_Sk', 'last_Yds_lost', 'last_p_Lng', 'last_p_Rate',
       'last_ru_Att', 'last_ru_Yds', 'last_ru_TD', 'last_ru_Lng',
       'last_re_targ', 'last_Rec', 'last_re_Yds', 'last_re_TD', 'last_re_Lng',
       'last_fmb', 'last_fmb_loss', 'lat', 'long', 'elevation']
# , 'lat', 'long', 'elevation'
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[VocabularyListCategoricalColumn(key='Date', vocabulary_list=('10/8/2020', '10/1/1979', '9/30/1990', '12/6/1998', '11/12/1978', '11/3/2013', '11/15/1970', '9/30/2012', '9/11/1988', '11/4/2007', '1/2/2005', '12/31/2017', '12/10/1989', '11/6/1983', '10/14/2012', '9/25/2011', '12/26/1982', '10/9/2016', '9/16/1979', '1/8/1995', '9/7/2014', '11/12/1972', '9/10/1989', '9/6/2007', '12/30/1979', '10/8/1978', '9/9/2018', '9/6/1992', '9/7/1986', '12/11/1994', '11/18/1979', '12/23/2001', '9/9/2019', '11/27/1983', '12/18/1983', '12/3/1989', '12/10/1972', '11/24/1996', '10/6/2002', '9/29/1991', '11/1/1998', '11/25/2007', '1/15/1995', '10/5/1997', '12/9/1973', '12/8/2019', '11/21/1993', '9/21/2014', '9/9/2007', '10/2/2005', '12/26/1999', '12/19/2004', '12/22/2019', '12/18/1988', '10/16/2011', '10/17/1993', '9/3/1995', '12/15/1985', '9/16/2018', '9/12/1982', '9/26/1988', '11/23/1970', '9/29/1968', '11/21/1999', '11/9/1997', '9/8/2008', '12/4/2016', '9/9/2012', '10/13/1996', '10/16/1983', '10/20/1996'

In [148]:
def make_input_fn(data_df, label_df, num_epochs=15, shuffle=True, batch_size=320):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)

In [149]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# We create a linear estimtor by passing the feature columns we created earlier

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpk5cykl2s', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [150]:
linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model

0.70068425


In [ ]:
pred_dicts = list(linear_est.predict(eval_input_fn))
probs = pd.Series([pred['probabilities'][1] for pred in pred_dicts])

probs.plot(kind='hist', bins=20, title='predicted probabilities')

In [ ]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[55])
print(result[55]['probabilities'][1])

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1c6_krj5/model.ckpt-3840
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Date                     12/3/1972
Tm               Minnesota Vikings
temp                            14
wind                            84
avg_p_Cmp                  16.0833
avg_p_Att                  27.6667
avg_p_Yds                  199.083
avg_p_TD                   1.33333
avg_Int                   0.666667
avg_Sk                           0
avg_Yds_lost                     0
avg_p_Lng                        0
avg_p_Rate                 88.6333
avg_ru_Att                 27.0833
avg_ru_Yds                   103.4
avg_ru_TD                      0.7
avg_ru_Lng                       0
avg_re_targ                      0
avg_Rec                    14.2333
avg_re_Yds                   203.1
avg_re_TD                      1.7
avg_re_Lng                     

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# Compute fpr, tpr, thresholds and roc auc
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_auc = auc(y_true, y_score)

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate or (1 - Specifity)')
plt.ylabel('True Positive Rate or (Sensitivity)')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")

NameError: ignored

In [ ]:
print(dfeval)
print(result[1:5115])

            Date                    Tm  ...  last_fmb  last_fmb_loss
0     10/26/1969   Pittsburgh Steelers  ...         0              0
1      9/29/1991  Tampa Bay Buccaneers  ...         0              0
2      9/26/1999      Tennessee Titans  ...         0              0
3     11/25/2019      Baltimore Ravens  ...         0              0
4      9/14/1980        Miami Dolphins  ...         0              0
...          ...                   ...  ...       ...            ...
5110  12/28/2008     Minnesota Vikings  ...         0              0
5111  10/18/2020      Cleveland Browns  ...         1              0
5112  11/19/2017        Houston Texans  ...         2              2
5113  10/23/2011    New Orleans Saints  ...         1              1
5114    9/6/1998         Detroit Lions  ...         0              0

[5115 rows x 44 columns]
[{'logits': array([-11.541704], dtype=float32), 'logistic': array([9.716315e-06], dtype=float32), 'probabilities': array([9.999902e-01, 9.716220e-

In [ ]:
dfeval.to_csv('dfevalfinal.csv')
list(result)

TypeError: ignored

In [ ]:
results = list(result)

In [ ]:
resdf = pd.DataFrame.from_dict(results)

In [ ]:
problist = []

for l in resdf['probabilities']:
  problist.append(l[1])

In [ ]:
from pandas import DataFrame
df = DataFrame (problist,columns=['Column_Name'])

In [ ]:
df.to_csv('predict.csv')